<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/embedding_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Row Names,instance_auroc,full_pixel_auroc,anomaly_pixel_auroc

mvtec_bottle,1.0,0.9848388223156217,0.9795612113551637

mvtec_cable,0.9968140929535232,0.9841372038295886,0.9752826952461647

mvtec_capsule,0.9792580773833267,0.9895443632504134,0.9874208674147473

mvtec_carpet,0.988362760834671,0.990496871831108,0.9878582223166695

mvtec_grid,0.9791144527986633,0.988007840848073,0.9836616355360858

mvtec_hazelnut,1.0,0.9867676790989195,0.9791327092006923

mvtec_leather,1.0,0.9928831713401822,0.9903851327309942

mvtec_metal_nut,0.9990224828934506,0.9834029444579608,0.979223844081265

mvtec_pill,0.9667212220403709,0.9780118319383861,0.9760501580403979

mvtec_screw,0.9877023980323837,0.9952476847840384,0.9938280036555261

mvtec_tile,0.9949494949494949,0.957116990925436,0.9407954758215441

mvtec_toothbrush,1.0,0.9857315428686503,0.9799756240845601

mvtec_transistor,0.9987499999999999,0.9609810989424673,0.9274338615727747

mvtec_wood,0.9912280701754386,0.9507743250682259,0.9373950548422444

mvtec_zipper,0.9950105042016807,0.9886782466534234,0.9856368251785296

Mean,0.9917955704175335,0.9811080412101663,0.9735760880718238


In [ ]:
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.utils.data
from torch import nn
from torchvision import models
import sys, os
# print(os.getcwd())
sys.path.append('./src')
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import patchcore
from patchcore import patchcore as patchcore_model
from siamese_full import SiameseNetwork
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torchvision import transforms
from typing import Callable, Any
import glob, os
import random

class Custom_Dataset_All(Dataset):
    """
    URL = https://www.robots.ox.ac.uk/~vgg/data/pets/
    """
    def __init__(self,
                 image_folder: str = r"D:xx\mvtech_anomaly_detection\mvtech_anomaly_detection",
                 image_name: str = "bottle",
                 split: str = "train",
                 transform: Callable = None,
                **kwargs):
        self._sub_dataset = ('bottle',  'cable',  'capsule',  'carpet',  'grid', 'hazelnut', 'leather',  'metal_nut',  'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper')
        self.image_name = image_name
        self.img_folder = image_folder
        self.transforms = transform
        self.imgs_pos, self.imgs_neg = self.get_image_paths(img_folder=image_folder)

    def get_image_paths(self, img_folder, split: str = "train"):
        img_positive = []
        img_negative = []
        # Define the file extensions for images
        image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.gif', '*.bmp', '*.tiff')
        # Initialize an empty list to store image paths
        for img_name in self._sub_dataset:
            for ext in image_extensions:
                img_fold = os.path.join(img_folder, img_name, "train", "good")
                if (img_name == self.image_name):
                    img_positive.extend(glob.glob(os.path.join(img_fold, ext)))
                else:
                    img_negative.extend(glob.glob(os.path.join(img_fold, ext)))

        img_positive = img_positive[:int(len(img_positive) * 0.75)] if split == "train" else img_positive[int(len(img_positive) * 0.75):]
        img_positive = [self.pil_loader(x) for x in img_positive]
        img_negative = [self.pil_loader(x) for x in img_negative]
        random.shuffle(img_positive)
        random.shuffle(img_negative)
        return img_positive, img_negative

    def __len__(self):
        return len(self.imgs_pos)

    def pil_loader(self, path: str) -> Image.Image:
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, "rb") as f:
            img = Image.open(f).convert("RGB")

            if self.transforms is not None:
                img = self.transforms(img)
            return img

    def default_loader(self, path: str) -> Any:
            return self.pil_loader(path)

    def __getitem__(self, idx):
        valid_indices = [i for i in range(len(self.imgs_pos)) if i != idx]
        random_index = random.choice(valid_indices)
        img_anchor = self.imgs_pos[idx]
        img_pos = self.imgs_pos[random_index]
        img_neg = self.imgs_neg[random.randint(0, len(self.imgs_neg) - 1)]
        return img_anchor, img_pos,  img_neg

def _standard_patchcore(image_dimension):
    patchcore_instance = patchcore_model.PatchCore(torch.device("cpu"))
    backbone = models.wide_resnet50_2(pretrained=False)
    backbone.name, backbone.seed = "wideresnet50", 0
    patchcore_instance.load(
        backbone=backbone,
        layers_to_extract_from=["layer2", "layer3"],
        device=torch.device("cpu"),
        input_shape=[3, image_dimension, image_dimension],
        pretrain_embed_dimension=1024,
        target_embed_dimension=1024,
        patchsize=3,
        patchstride=1,
        spade_nn=2,
        sub_model=None
    )
    return patchcore_instance

if __name__ == "__main__":
    epoch_num = 100
    batch_size = 8
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    network = SiameseNetwork(in_dimension=3).to(device='cuda')
    criterion = nn.TripletMarginLoss()
    criterion_2 = nn.MSELoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
    transfomation = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

        ])
    _dataset = Custom_Dataset_All(transform=transfomation)
    _data_loader = DataLoader(_dataset, batch_size=batch_size, shuffle=True)

    _dataset_test = Custom_Dataset_All(transform=transfomation, split="test")
    _test_data_loader = DataLoader(_dataset_test, batch_size=batch_size, shuffle=True)
    mean, std = 0, 0.2
    # model = _standard_patchcore(224)
    for epoch in range(epoch_num):
        print(f"=========== epoch {epoch} ===========")
        loss_item = 0.
        valid_loss = 0.
        network.train()
        for i, image in enumerate(_data_loader):
            if (image[0].shape[0] < batch_size): continue
            # image = torch.concat(image, dim=0)
            # feature = model.get_feature(image)
            bs, c, h, w = image[0].shape
            gaussian_noise = torch.randn(1, c, h, w) * std + mean + image[0]
            img0, img1, img2 = image[0], image[1], gaussian_noise
            img0, img1, img2 = img0.cuda(), img1.cuda(), img2.cuda()

            optimizer.zero_grad()
            output1, output2, output3, out_noise = network(img0, img1, img2)
            loss = criterion_2(img0, out_noise) + criterion(output1, output2, output3)
            loss.backward()
            optimizer.step()
            loss_item += loss.item()
            if i % 1 == 0:
                print(f"Epoch [{epoch+1}/{epoch_num}], Step [{i}/{len(_data_loader)}], Loss: {(loss_item/(i+1)):.4f}")


        network.eval()
        with torch.no_grad():
            for i, image in enumerate(_test_data_loader):
                if (image[0].shape[0] < 2): continue
                img0, img1, img2 = image[0], image[1], image[2]
                img0, img1, img2 = img0.cuda(), img1.cuda(), img2.cuda()

                output1, output2, output3, out_noise = network(img0, img1, img2)
                loss = criterion_2(img0, out_noise) + criterion(output1, output2, output3)
                valid_loss += loss.item()
            print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {(valid_loss/(i+1)):.4f}")


    torch.save(network.state_dict(), f'./sub_model/siamese_wide_resnet50_2_final.pth')

In [ ]:
"""PatchCore and PatchCore detection methods."""
import logging
import os
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import tqdm

import patchcore
import patchcore.backbones
import patchcore.common
import patchcore.sampler
from patchcore.siamese import SiameseNetwork

LOGGER = logging.getLogger(__name__)


class PatchCore(torch.nn.Module):
    def __init__(self, device):
        """PatchCore anomaly detection class."""
        super(PatchCore, self).__init__()
        self.device = device

    def load(
        self,
        backbone,
        layers_to_extract_from,
        device,
        input_shape,
        pretrain_embed_dimension,
        target_embed_dimension,
        patchsize=3,
        patchstride=1,
        anomaly_score_num_nn=1,
        featuresampler=patchcore.sampler.IdentitySampler(),
        nn_method=patchcore.common.FaissNN(False, 4),
        sub_model=True,
        **kwargs,
    ):
        self.backbone = backbone.to(device)
        self.layers_to_extract_from = layers_to_extract_from
        self.input_shape = input_shape

        self.device = device
        self.patch_maker = PatchMaker(patchsize, stride=patchstride)

        self.forward_modules = torch.nn.ModuleDict({})

        feature_aggregator = patchcore.common.NetworkFeatureAggregator(
            self.backbone, self.layers_to_extract_from, self.device
        )
        feature_dimensions = feature_aggregator.feature_dimensions(input_shape)
        self.forward_modules["feature_aggregator"] = feature_aggregator

        preprocessing = patchcore.common.Preprocessing(
            feature_dimensions, pretrain_embed_dimension
        )
        self.forward_modules["preprocessing"] = preprocessing

        self.target_embed_dimension = target_embed_dimension
        preadapt_aggregator = patchcore.common.Aggregator(
            target_dim=target_embed_dimension
        )

        _ = preadapt_aggregator.to(self.device)

        self.forward_modules["preadapt_aggregator"] = preadapt_aggregator

        self.anomaly_scorer = patchcore.common.NearestNeighbourScorer(
            n_nearest_neighbours=anomaly_score_num_nn, nn_method=nn_method
        )

        self.anomaly_segmentor = patchcore.common.RescaleSegmentor(
            device=self.device, target_size=input_shape[-2:]
        )

        self.featuresampler = featuresampler

        if sub_model:
            self.sub_model = SiameseNetwork(in_dimension=1536, out_dimension=target_embed_dimension).to(device=device)
            self.sub_model.load_state_dict(torch.load('./sub_model/siamese_wide_resnet50_2_final.pth'))

    def embed(self, data):
        if isinstance(data, torch.utils.data.DataLoader):
            features = []
            for image in data:
                if isinstance(image, dict):
                    image = image["image"]
                with torch.no_grad():
                    input_image = image.to(torch.float).to(self.device)
                    features.append(self._embed(input_image))
            return features
        return self._embed(data)

    def _embed(self, images, detach=True, provide_patch_shapes=False):
        """Returns feature embeddings for images."""

        def _detach(features):
            if detach:
                return [x.detach().cpu().numpy() for x in features]
            return features

        _ = self.forward_modules["feature_aggregator"].eval()
        with torch.no_grad():
            features = self.forward_modules["feature_aggregator"](images)

        features = [features[layer] for layer in self.layers_to_extract_from]

        features = [
            self.patch_maker.patchify(x, return_spatial_info=True) for x in features
        ]
        patch_shapes = [x[1] for x in features]
        features = [x[0] for x in features]
        ref_num_patches = patch_shapes[0]

        for i in range(1, len(features)):
            _features = features[i]
            patch_dims = patch_shapes[i]

            # TODO(pgehler): Add comments
            _features = _features.reshape(
                _features.shape[0], patch_dims[0], patch_dims[1], *_features.shape[2:]
            )
            _features = _features.permute(0, -3, -2, -1, 1, 2)
            perm_base_shape = _features.shape
            _features = _features.reshape(-1, *_features.shape[-2:])
            _features = F.interpolate(
                _features.unsqueeze(1),
                size=(ref_num_patches[0], ref_num_patches[1]),
                mode="bilinear",
                align_corners=False,
            )
            _features = _features.squeeze(1)
            _features = _features.reshape(
                *perm_base_shape[:-2], ref_num_patches[0], ref_num_patches[1]
            )
            _features = _features.permute(0, -2, -1, 1, 2, 3)
            _features = _features.reshape(len(_features), -1, *_features.shape[-3:])
            features[i] = _features
        features = [x.reshape(-1, *x.shape[-3:]) for x in features]

        # As different feature backbones & patching provide differently
        # sized features, these are brought into the correct form here.
        features = self.forward_modules["preprocessing"](features)
        features = self.forward_modules["preadapt_aggregator"](features)

        if provide_patch_shapes:
            return _detach(features), patch_shapes
        return _detach(features)

    def fit(self, training_data):
        """PatchCore training.

        This function computes the embeddings of the training data and fills the
        memory bank of SPADE.
        """
        # self._fill_memory_bank(training_data)
        self._fill_memory_bank_with_sub_model(training_data)

    def get_feature(self, input_image):
        """Computes and sets the support features for SPADE."""

        _ = self.forward_modules.eval()
        with torch.no_grad():
            input_image = input_image.to(torch.float).to(self.device)
            _ = self.forward_modules["feature_aggregator"].eval()
            features = self.forward_modules["feature_aggregator"](input_image)
            features = [features[layer] for layer in self.layers_to_extract_from]
            return features

    def _fill_memory_bank(self, input_data):
        """Computes and sets the support features for SPADE."""
        _ = self.forward_modules.eval()

        def _image_to_features(input_image):
            with torch.no_grad():
                input_image = input_image.to(torch.float).to(self.device)
                return self._embed(input_image)

        features = []
        with tqdm.tqdm(
            input_data, desc="Computing support features...", position=1, leave=False
        ) as data_iterator:
            for image in data_iterator:
                if isinstance(image, dict):
                    image = image["image"]
                features.append(_image_to_features(image))

        features = np.concatenate(features, axis=0)
        features = self.featuresampler.run(features)

        self.anomaly_scorer.fit(detection_features=[features])

    def _fill_memory_bank_with_sub_model(self, input_data, detach=True, provide_patch_shapes=False):
        """Computes and sets the support features for SPADE."""

        def _detach(features):
            if detach:
                return [x.detach().cpu().numpy() for x in features]
            return features

        _ = self.forward_modules.eval()

        def _image_to_features(input_image):
            self.sub_model.eval()
            with torch.no_grad():
                input_image = input_image.to(torch.float).to(self.device)
                # out_feature = self.get_feature(input_image=input_image)
                # patch_shapes = [[x.shape[-2], x.shape[-1]] for x in out_feature]
                patch_shapes = [(28,28),(14,14)]
                # img0, img1 = out_feature[0].to(self.device), out_feature[1].to(self.device)
                out_feature = self.sub_model.forward_once(input_image)
                out_feature = out_feature.permute(0,2,3,1)
                out_feature = torch.reshape(out_feature, (-1, self.target_embed_dimension))

                if provide_patch_shapes:
                    return _detach(out_feature), patch_shapes
                return _detach(out_feature)

        def _image_to_features_raw(input_image):
            with torch.no_grad():
                input_image = input_image.to(torch.float).to(self.device)
                return self._embed(input_image)

        features = []
        with tqdm.tqdm(
            input_data, desc="Computing support features...", position=1, leave=False
        ) as data_iterator:
            for image in data_iterator:
                if isinstance(image, dict):
                    image = image["image"]
                features.append( _image_to_features(image))
                # features.append([(x - y)  for x, y in zip(_image_to_features_raw(image), _image_to_features(image))])

        features = np.concatenate(features, axis=0)
        features = self.featuresampler.run(features)

        self.anomaly_scorer.fit(detection_features=[features])


    def _embed_with_sub_model(self, input_data, detach=True, provide_patch_shapes=False):
        def _detach(features):
            if detach:
                return [x.detach().cpu().numpy() for x in features]
            return features
        def _image_to_features(input_image):
            self.sub_model.eval()
            with torch.no_grad():
                input_image = input_image.to(torch.float).to(self.device)
                patch_shapes = [(28,28),(14,14)]
                # out_feature = self.get_feature(input_image=input_image)
                # patch_shapes = [[x.shape[-2], x.shape[-1]] for x in out_feature]
                # img0, img1 = out_feature[0].to(self.device), out_feature[1].to(self.device)
                out_feature = self.sub_model.forward_once(input_image)
                out_feature = out_feature.permute(0,2,3,1)
                out_feature = torch.reshape(out_feature, (-1, self.target_embed_dimension))

                if provide_patch_shapes:
                    return _detach(out_feature), patch_shapes
                return _detach(out_feature)

        def _image_to_features_raw(input_image):
            with torch.no_grad():
                input_image = input_image.to(torch.float).to(self.device)
                return self._embed(input_image)

        return _image_to_features(input_image=input_data)

    def predict(self, data):
        if isinstance(data, torch.utils.data.DataLoader):
            return self._predict_dataloader(data)
        return self._predict(data)

    def _predict_dataloader(self, dataloader):
        """This function provides anomaly scores/maps for full dataloaders."""
        _ = self.forward_modules.eval()

        scores = []
        masks = []
        labels_gt = []
        masks_gt = []
        with tqdm.tqdm(dataloader, desc="Inferring...", leave=False) as data_iterator:
            for image in data_iterator:
                if isinstance(image, dict):
                    labels_gt.extend(image["is_anomaly"].numpy().tolist())
                    masks_gt.extend(image["mask"].numpy().tolist())
                    image = image["image"]
                _scores, _masks = self._predict(image)
                for score, mask in zip(_scores, _masks):
                    scores.append(score)
                    masks.append(mask)
        return scores, masks, labels_gt, masks_gt

    def _predict(self, images):
        """Infer score and mask for a batch of images."""
        images = images.to(torch.float).to(self.device)
        _ = self.forward_modules.eval()

        batchsize = images.shape[0]
        with torch.no_grad():
            # features, patch_shapes = self._embed(images, provide_patch_shapes=True)
            features, patch_shapes = self._embed_with_sub_model(images, provide_patch_shapes=True)

            # features = [(x - y) for x, y in zip(features_raw, features)]
            # features = [(x - y) * 2 for x, y in zip(features_raw, features)]
            features = np.asarray(features)

            patch_scores = image_scores = self.anomaly_scorer.predict([features])[0]
            image_scores = self.patch_maker.unpatch_scores(
                image_scores, batchsize=batchsize
            )
            image_scores = image_scores.reshape(*image_scores.shape[:2], -1)
            image_scores = self.patch_maker.score(image_scores)

            patch_scores = self.patch_maker.unpatch_scores(
                patch_scores, batchsize=batchsize
            )
            scales = patch_shapes[0]
            patch_scores = patch_scores.reshape(batchsize, scales[0], scales[1])

            masks = self.anomaly_segmentor.convert_to_segmentation(patch_scores)

        return [score for score in image_scores], [mask for mask in masks]

    @staticmethod
    def _params_file(filepath, prepend=""):
        return os.path.join(filepath, prepend + "patchcore_params.pkl")

    def save_to_path(self, save_path: str, prepend: str = "") -> None:
        LOGGER.info("Saving PatchCore data.")
        self.anomaly_scorer.save(
            save_path, save_features_separately=False, prepend=prepend
        )
        patchcore_params = {
            "backbone.name": self.backbone.name,
            "layers_to_extract_from": self.layers_to_extract_from,
            "input_shape": self.input_shape,
            "pretrain_embed_dimension": self.forward_modules[
                "preprocessing"
            ].output_dim,
            "target_embed_dimension": self.forward_modules[
                "preadapt_aggregator"
            ].target_dim,
            "patchsize": self.patch_maker.patchsize,
            "patchstride": self.patch_maker.stride,
            "anomaly_scorer_num_nn": self.anomaly_scorer.n_nearest_neighbours,
        }
        with open(self._params_file(save_path, prepend), "wb") as save_file:
            pickle.dump(patchcore_params, save_file, pickle.HIGHEST_PROTOCOL)

    def load_from_path(
        self,
        load_path: str,
        device: torch.device,
        nn_method: patchcore.common.FaissNN(False, 4),
        prepend: str = "",
    ) -> None:
        LOGGER.info("Loading and initializing PatchCore.")
        with open(self._params_file(load_path, prepend), "rb") as load_file:
            patchcore_params = pickle.load(load_file)
        patchcore_params["backbone"] = patchcore.backbones.load(
            patchcore_params["backbone.name"]
        )
        patchcore_params["backbone"].name = patchcore_params["backbone.name"]
        del patchcore_params["backbone.name"]
        self.load(**patchcore_params, device=device, nn_method=nn_method)

        self.anomaly_scorer.load(load_path, prepend)


# Image handling classes.
class PatchMaker:
    def __init__(self, patchsize, stride=None):
        self.patchsize = patchsize
        self.stride = stride

    def patchify(self, features, return_spatial_info=False):
        """Convert a tensor into a tensor of respective patches.
        Args:
            x: [torch.Tensor, bs x c x w x h]
        Returns:
            x: [torch.Tensor, bs * w//stride * h//stride, c, patchsize,
            patchsize]
        """
        padding = int((self.patchsize - 1) / 2)
        unfolder = torch.nn.Unfold(
            kernel_size=self.patchsize, stride=self.stride, padding=padding, dilation=1
        )
        unfolded_features = unfolder(features)
        number_of_total_patches = []
        for s in features.shape[-2:]:
            n_patches = (
                s + 2 * padding - 1 * (self.patchsize - 1) - 1
            ) / self.stride + 1
            number_of_total_patches.append(int(n_patches))
        unfolded_features = unfolded_features.reshape(
            *features.shape[:2], self.patchsize, self.patchsize, -1
        )
        unfolded_features = unfolded_features.permute(0, 4, 1, 2, 3)

        if return_spatial_info:
            return unfolded_features, number_of_total_patches
        return unfolded_features

    def unpatch_scores(self, x, batchsize):
        return x.reshape(batchsize, -1, *x.shape[1:])

    def score(self, x):
        was_numpy = False
        if isinstance(x, np.ndarray):
            was_numpy = True
            x = torch.from_numpy(x)
        while x.ndim > 1:
            x = torch.max(x, dim=-1).values
        if was_numpy:
            return x.numpy()
        return x


In [ ]:
import torch
from torch import nn

from torchsummary import summary

class SiameseNetwork(nn.Module):
    def __init__(self, in_dimension=3, out_dimension=1024, device='cuda'):
        super(SiameseNetwork, self).__init__()

        self.out_dimension = out_dimension
        self.in_dimension = in_dimension
        self.device = device

        self.fcn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # nn.Conv2d(32, 64, 3, 2),
            # nn.BatchNorm2d(64),
            # nn.ReLU(),

            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # nn.Conv2d(128, 256, 3, 2),
            # nn.BatchNorm2d(256),
            # nn.ReLU(),

            nn.Conv2d(128, 512, 3, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 1024, 3, padding='same'),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
        )
        self.mean = nn.Parameter(torch.randn(1).to(device=device))
        self.std = nn.Parameter(torch.randn(1).to(device=device))
        self.linear_sub_1 = nn.LazyLinear(out_features=256)
        self.linear_sub_2 = nn.LazyLinear(out_features=1024)
        self.global_pooling = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()


    def forward_once(self, x, add_noise=False):
    # def forward(self, x):
        # Forward pass
        if add_noise:
            bs, c, w, h = x.shape
            gaussian_noise = torch.randn(1, c, w, h).to(device=self.device) * self.std + self.mean
            x = x + gaussian_noise
            out_noise = x

        x = self.fcn(x)

        global_pooling = self.global_pooling(x)

        attention_score = torch.transpose(global_pooling, -3, -1)
        attention_score = self.linear_sub_1(attention_score)
        attention_score = self.relu(attention_score)
        attention_score = self.linear_sub_2(attention_score)
        attention_score = self.sigmoid(attention_score)
        attention_score = torch.transpose(attention_score, -3, -1)

        output = torch.mul(x, attention_score)
        if add_noise:
            return output, out_noise
        # output = torch.reshape(output, (-1, self.out_dimension))
        return output

    # def forward(self, input1, input2):
    #     # forward pass of input 1
    #     output1 = self.forward_once(torch.unsqueeze(input1[0,:,:,:], dim=0), torch.unsqueeze(input2[0,:,:,:], dim=0))
    #     # forward pass of input 2
    #     output2 = self.forward_once(torch.unsqueeze(input1[1,:,:,:], dim=0), torch.unsqueeze(input2[1,:,:,:], dim=0))
    #     return output1, output2

    def forward(self, input1, input2, input3):
        # forward pass of input 1
        output1, output_noise = self.forward_once(input1, add_noise=True)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        # forward pass of input 2
        output3 = self.forward_once(output_noise)
        return output1, output2, output3, output_noise


# network = SiameseNetwork(3, 1024).to(device='cuda')
# summary(network, ((3, 224, 224)))
# tensor = torch.randn(4, 3, 224, 224).to(device='cuda')
# print(network(tensor).shape)